In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from arch import arch_model

# Download and align data
start_date = '2005-01-01'
end_date = '2025-01-01'

# Fetch data with identical dates
vix = yf.download('^VIX', start=start_date, end=end_date, auto_adjust=True)
sp500 = yf.download('^GSPC', start=start_date, end=end_date, auto_adjust=True)

# Merge to ensure date alignment
data = pd.merge(sp500[['Close']], vix[['Close']], left_index=True, right_index=True, 
                suffixes=('_sp500', '_vix'))
print(f"Original data size: {len(data)}")
print(data.head())
# Calculate log returns
data['log_ret_sp500'] = np.log(data['Close_sp500'] / data['Close_sp500'].shift(1))
data['log_ret_vix'] = np.log(data['Close_vix'] / data['Close_vix'].shift(1))
data.dropna(inplace=True)
print(f"Aligned data size after dropna: {len(data)}")

# Fit GARCH on S&P500 log returns
garch_model = arch_model(
    data['log_ret_sp500'], 
    mean='Constant',  # Explicitly model mean
    vol='GARCH', 
    p=1, 
    q=1
)
garch_result = garch_model.fit(update_freq=0, disp='off')
print(garch_result.summary())

# Get standardized residuals
data['garch_resid'] = garch_result.std_resid

# Split data before scaling (avoid leakage)
split_idx = int(0.6 * len(data))
train_data = data.iloc[:split_idx]
test_data = data.iloc[split_idx:]

# Scale VIX log returns using training data only
vix_scaler = StandardScaler()
train_data['log_ret_vix_scaled'] = vix_scaler.fit_transform(train_data[['log_ret_vix']])
test_data['log_ret_vix_scaled'] = vix_scaler.transform(test_data[['log_ret_vix']])

# Create sequences [GARCH residual, Scaled VIX return] -> next VIX return
SEQ_LEN = 30

def create_sequences(df):
    xs, ys, prices = [], [], []
    for i in range(SEQ_LEN, len(df)):
        xs.append(df[['garch_resid', 'log_ret_vix_scaled']].iloc[i-SEQ_LEN:i].values)
        ys.append(df['log_ret_vix_scaled'].iloc[i])
        prices.append(df['Close_vix'].iloc[i-1])  # Previous close for price conversion
    return np.array(xs), np.array(ys), np.array(prices)

X_train, y_train, train_prev_prices = create_sequences(train_data)
X_test, y_test, test_prev_prices = create_sequences(test_data)

print(f"Train sequences: {X_train.shape}, Test sequences: {X_test.shape}")

# PyTorch Dataset
class FinanceDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_ds = FinanceDataset(X_train, y_train)
test_ds = FinanceDataset(X_test, y_test)
train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=64, shuffle=False)

# LSTM Model
class LSTMModel(nn.Module):
    def __init__(self, input_size=2, hidden_size=64, num_layers=2):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, 1)
    
    def forward(self, x):
        x, _ = self.lstm(x)
        x = x[:, -1]  # Last timestep
        return self.linear(x)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = LSTMModel().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Training
EPOCHS = 50
for epoch in range(EPOCHS):
    model.train()
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch).squeeze()
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch {epoch+1}/{EPOCHS} Loss: {loss.item():.6f}')

# Evaluation
model.eval()
with torch.no_grad():
    # Predict log returns
    test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
    preds_scaled = model(test_tensor).cpu().numpy().squeeze()
    
    # Inverse transform to actual log returns
    pred_log_ret = vix_scaler.inverse_transform(preds_scaled.reshape(-1, 1)).flatten()
    true_log_ret = vix_scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()
    
    # Convert log returns to prices
    pred_prices = test_prev_prices * np.exp(pred_log_ret)
    true_prices = test_data['Close_vix'].iloc[SEQ_LEN:].values  # Actual target prices
    
    # Calculate metrics
    logret_r2 = r2_score(true_log_ret, pred_log_ret)
    logret_rmse = np.sqrt(mean_squared_error(true_log_ret, pred_log_ret))
    price_rmse = np.sqrt(mean_squared_error(true_prices, pred_prices))

print("\nLog Return Metrics:")
print(f"R²: {logret_r2:.4f}, RMSE: {logret_rmse:.6f}")
print("\nPrice Metrics:")
print(f"Price RMSE: {price_rmse:.4f}")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Original data size: 5033
Price             Close       
Ticker            ^GSPC   ^VIX
Date                          
2005-01-03  1202.079956  14.08
2005-01-04  1188.050049  13.98
2005-01-05  1183.739990  14.09
2005-01-06  1187.890015  13.58
2005-01-07  1186.189941  13.49


KeyError: 'Close_sp500'